In [196]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import rasterio

In [197]:
def maxwind_to_geotiff_NL(model,fc_time):
    
    #set paths
    forecast_path = os.path.join("C:\\Data",'forecast')
    
    if model == 'GFS':
        GFS = os.path.join(forecast_path,"GRIB","gfs.t00z.pgrb2.0p25.f{}".format(fc_time))

        ds = xr.load_dataset(GFS, filter_by_keys={'typeOfLevel': 'maxWind'},engine="cfgrib")
        ds['maxwind'] = (np.sqrt(np.square(ds['u'])+np.square(ds['v'])))*1.852
        ds['longitude'] = (ds['longitude'] + 180) % 360 - 180
        
    elif model == 'ECMWF':
        ECMWF = os.path.join(forecast_path,"GRIB","ECMD_EUR_202202170000_{}00_GB".format(fc_time))
        ds = xr.load_dataset(ECMWF,engine="cfgrib")

        ds['maxwind'] = ds['fg10']*3.6    
    
    
    min_lon = 3
    min_lat = 50
    max_lon = 8
    max_lat = 54

    mask_lon = (ds.longitude >= min_lon) & (ds.longitude <= max_lon)
    mask_lat = (ds.latitude >= min_lat) & (ds.latitude <= max_lat)

    cropped_ds = ds.where(mask_lon & mask_lat, drop=True)

    width = len(cropped_ds.longitude)
    height = len(cropped_ds.latitude)
    
    cropped_ds['maxwind'].rio.to_raster(os.path.join(forecast_path,'GeoTIFF',"{}_f{}_NL.tif".format(model,fc_time)),crs="epsg:4326")

    with rasterio.open(os.path.join(forecast_path,'GeoTIFF',"{}_f{}_NL.tif".format(model,fc_time))) as src:
        affine = rasterio.transform.from_bounds(min_lon, min_lat, max_lon, max_lat, width, height)
        array = src.read(1)

    with rasterio.open(
        os.path.join(forecast_path,'GeoTIFF',"{}_f{}_NL.tif".format(model,fc_time)),    'w',    driver='GTiff',    height=array.shape[0],    width=array.shape[1],    count=1,
        dtype=array.dtype,nodata=9999,
        crs='epsg:4326',  transform=affine) as dst:
        dst.write(array, 1)     

In [163]:
for fc_time in ['036','037','038','039','040','041','042','043','044','045','046']:
    maxwind_to_geotiff_NL(model='ECMWF',fc_time=fc_time)

Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_03600_GB.923a8.idx' incompatible with GRIB file
Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_03700_GB.923a8.idx' incompatible with GRIB file
Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_03800_GB.923a8.idx' incompatible with GRIB file
Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_03900_GB.923a8.idx' incompatible with GRIB file
Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_04000_GB.923a8.idx' incompatible with GRIB file
Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_04100_GB.923a8.idx' incompatible with GRIB file
Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_04200_GB.923a8.idx' incompatible with GRIB file


In [199]:
maxwind_to_geotiff_NL(model='GFS',fc_time='018')

In [ ]:
    filter_by_keys={'typeOfLevel': 'heightAboveGround'}
    filter_by_keys={'typeOfLevel': 'isobaricInhPa'}
    filter_by_keys={'typeOfLevel': 'hybrid'}
    filter_by_keys={'typeOfLevel': 'heightAboveSea'}
    filter_by_keys={'typeOfLevel': 'entireAtmosphere'}
    filter_by_keys={'typeOfLevel': 'nominalTop'}

In [192]:
fc_time='024'
HAR = os.path.join(forecast_path,"HAP2_N25_202202171200_{}00_GB".format(fc_time))
ds = xr.load_dataset(HAR,filter_by_keys={'typeOfLevel': 'entireAtmosphere'},engine="cfgrib")
ds

Ignoring index file 'C:\\Data\\forecast\\GRIB\\HAP2_N25_202202171200_02400_GB.923a8.idx' incompatible with GRIB file


<xarray.Dataset>
Dimensions:           (latitude: 300, longitude: 300)
Coordinates:
    time              datetime64[ns] 2022-02-17T12:00:00
    step              timedelta64[ns] 1 days
    entireAtmosphere  float64 0.0
  * latitude          (latitude) float64 49.0 49.02 49.05 ... 55.83 55.85 55.88
  * longitude         (longitude) float64 0.0 0.037 0.074 ... 10.99 11.03 11.06
    valid_time        datetime64[ns] 2022-02-18T12:00:00
Data variables:
    unknown           (latitude, longitude) float32 3.082e+03 ... 2.807e+03
Attributes:
    GRIB_edition:            1
    GRIB_centre:             99
    GRIB_centreDescription:  DeBilt, Netherlands
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             DeBilt, Netherlands
    history:                 2022-02-17T20:42 GRIB to CDM+CF via cfgrib-0.9.1...

In [195]:
fc_time = '040'
ECMWF = os.path.join(forecast_path,"ECMD_EUR_202202170000_{}00_GB".format(fc_time))
ds = xr.load_dataset(ECMWF,engine="cfgrib")
ds

Ignoring index file 'C:\\Data\\forecast\\GRIB\\ECMD_EUR_202202170000_04000_GB.923a8.idx' incompatible with GRIB file


<xarray.Dataset>
Dimensions:        (latitude: 251, longitude: 501, isobaricInhPa: 5)
Coordinates:
    number         int32 0
    time           datetime64[ns] 2022-02-17
    step           timedelta64[ns] 1 days 16:00:00
    surface        float64 0.0
  * latitude       (latitude) float64 62.0 61.9 61.8 61.7 ... 37.2 37.1 37.0
  * longitude      (longitude) float64 -30.0 -29.9 -29.8 ... 19.8 19.9 20.0
    valid_time     datetime64[ns] 2022-02-18T16:00:00
  * isobaricInhPa  (isobaricInhPa) float64 950.0 850.0 700.0 500.0 300.0
Data variables: (12/30)
    skt            (latitude, longitude) float32 278.2 278.3 ... 288.7 288.6
    hcc            (latitude, longitude) float32 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    sf             (latitude, longitude) float32 0.003153 0.00316 ... 0.0 0.0
    lcc            (latitude, longitude) float32 0.3828 0.4297 ... 0.007812 0.0
    cape           (latitude, longitude) float32 1.469 2.793 2.168 ... 0.0 0.0
    sd             (latitude, longitude) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...             ...
    cbh            (latitude, longitude) float32 829.7 830.0 830.0 ... nan nan
    e              (latitude, longitude) float32 -0.006958 ... -0.004535
    q              (isobaricInhPa, latitude, longitude) float32 0.002093 ... ...
    mx2t           (latitude, longitude) float32 274.4 274.4 ... 288.1 288.0
    mn2t           (latitude, longitude) float32 274.2 274.2 ... 288.0 288.0
    licd           (latitude, longitude) float32 -4.366e-11 ... -4.366e-11
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-02-17T20:45 GRIB to CDM+CF via cfgrib-0.9.1...

In [190]:
ds['unknown'][1]

<xarray.DataArray 'unknown' ()>
array(-18.596054, dtype=float32)
Coordinates:
    time               datetime64[ns] 2022-02-17T12:00:00
    step               timedelta64[ns] 1 days
    heightAboveGround  float64 10.0
    valid_time         datetime64[ns] 2022-02-18T12:00:00